## Spark DataFrames

DataFrames are immutable distributed collection of data that is organized like a table in relational database. Spark DataFrames are similar to pandas dataframes. 

In Spark 2.0 we are using `SparkSession` instead of SQLContext to work with DataFrames. The various Spark contexts like HiveContext, SQLContext, StreamingContext and SparkContext are merged together into SparkSession.

### Catalyst Optimizer

Catalyst Optimizer is similar to DAG scheduler in execution or RDD. Catalyst Optimizer finds the most efficient plan to execute data operations.

As opposed to immediately processing the query, the spark engine's Catalyst Optimizer compiles and optimizes logical plan (series of algebraic and logical constructs like SELECT, GROUPBY represented in tree like structure) and has a cost optimizer that determines h\the most efficient physical plan (contains series of low level functions represented as a tree like structure) generated.

- It helps to requce the query time.

- It is written in scala and as a developer you can write you own optimization functions.

### Why DataFrames are faster than RDDs

- RDDs are not optimized as DataFrames optimized with Catalyst Optimizer.

- Communication between python and JVM made RDDs very slow.

### Creating a DataFrame

Typically, you will create DataFrames by importing data using SparkSession.

In [2]:
# creating a JSON file
# creating RDD out of this JSON file
JsonRDD = sc.parallelize(("""
{ "id": "123",
"name": "kalyan",
"age" : 25,
"eyeColor" : "brown"}
""",
                         
"""
{ "id": "124",
"name": "Varun",
"age" : 30,
"eyeColor" : "black"}
""",
                         
"""
{ "id": "125",
"name": "Shiva",
"age" : 35,
"eyeColor" : "blue"}
"""))

In [6]:
# convert this RDD into DataFrame by using SparkSession read.json method

from pyspark.sql import SparkSession
spark = SparkSession(sc)


df = spark.read.json(JsonRDD)
df.show()

+---+--------+---+------+
|age|eyeColor| id|  name|
+---+--------+---+------+
| 25|   brown|123|kalyan|
| 30|   black|124| Varun|
| 35|    blue|125| Shiva|
+---+--------+---+------+



In [8]:
# to execute SQL queries create temporary table first
df.createOrReplaceTempView("df")
spark.sql("select * from df").collect()

[Row(age=25, eyeColor='brown', id='123', name='kalyan'),
 Row(age=30, eyeColor='black', id='124', name='Varun'),
 Row(age=35, eyeColor='blue', id='125', name='Shiva')]

#### Getting Number of Rows in a DataFrame

In [10]:
# No. of Rows
df.count()

3

#### Running Filter statements

In [12]:
# get id, age where age = 30

df.select("id", "age").filter("age = 30").show()

+---+---+
| id|age|
+---+---+
|124| 30|
+---+---+



In [13]:
# Another way

df.select(df.id, df.age).filter(df.age == 30).show()

+---+---+
| id|age|
+---+---+
|124| 30|
+---+---+



In [17]:
# wildcards

df.select(df.name).filter("eyeColor like 'bla%'").show()

+-----+
| name|
+-----+
|Varun|
+-----+



#### Querying dataframe with SQL statement?

In [18]:
spark.sql("select count(*) from df").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



#### Running filter statements using where clauses

In [19]:
spark.sql("select id, name from df where age = 30").show()

+---+-----+
| id| name|
+---+-----+
|124|Varun|
+---+-----+

